In [1]:
import pandas as pd
import numpy as np
import scipy.io
import h5py
import mat73

from tqdm import tqdm

from pynwb import NWBHDF5IO

import pyarrow as pa
import pyarrow.parquet as pq

from pyspark.sql import SparkSession
from pyspark.sql import functions as F

In [2]:
file = h5py.File('data/000070/sub-Jenkins/sub-Jenkins_ses-20090918_behavior+ecephys.nwb')
io = NWBHDF5IO(file=file, mode='r', load_namespaces=True)

FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = 'data/000070/sub-Jenkins/sub-Jenkins_ses-20090918_behavior+ecephys.nwb', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [53]:
nwb = io.read()
nwb

,x,y,z,imp,location,filtering,group,group_name,gain,offset
id,,,,,,,,,,
1,NaN,NaN,NaN,-1.0,"Pre-Motor Cortex, dorsal",1000Hz,"1 pynwb.ecephys.ElectrodeGroup at 0x11595178320\nFields:\n description: array corresponding to device implanted at PMd\n device: Utah Array(PMd) pynwb.device.Device at 0x10977868304\nFields:\n description: 96 channel utah array\n manufacturer: BlackRock Microsystems\n\n location: Caudal, dorsal Pre-motor cortex, Left hemisphere\n",1,1.0,0.0
2,NaN,NaN,NaN,-1.0,"Pre-Motor Cortex, dorsal",1000Hz,"1 pynwb.ecephys.ElectrodeGroup at 0x11595178320\nFields:\n description: array corresponding to device implanted at PMd\n device: Utah Array(PMd) pynwb.device.Device at 0x10977868304\nFields:\n description: 96 channel utah array\n manufacturer: BlackRock Microsystems\n\n location: Caudal, dorsal Pre-motor cortex, Left hemisphere\n",1,1.0,0.0
3,NaN,NaN,NaN,-1.0,"Pre-Motor Cortex, dorsal",1000Hz,"1 pynwb.ecephys.ElectrodeGroup at 0x11595178320\nFields:\n description: array corresponding to device implanted at PMd\n device: Utah Array(PMd) pynwb.device.Device at 0x10977868304\nFields:\n description: 96 channel utah array\n manufacturer: BlackRock Microsystems\n\n location: Caudal, dorsal Pre-motor cortex, Left hemisphere\n",1,1.0,0.0
4,NaN,NaN,NaN,-1.0,"Pre-Motor Cortex, dorsal",1000Hz,"1 pynwb.ecephys.ElectrodeGroup at 0x11595178320\nFields:\n description: array corresponding to device implanted at PMd\n device: Utah Array(PMd) pynwb.device.Device at 0x10977868304\nFields:\n description: 96 channel utah array\n manufacturer: BlackRock Microsystems\n\n location: Caudal, dorsal Pre-motor cortex, Left hemisphere\n",1,1.0,0.0
,x,y,z,imp,location,filtering,group,group_name,gain,offset
id,,,,,,,,,,
1,NaN,NaN,NaN,-1.0,"Pre-Motor Cortex, dorsal",1000Hz,"1 pynwb.ecephys.ElectrodeGroup at 0x11595178320\nFields:\n description: array corresponding to device implanted at PMd\n device: Utah Array(PMd) pynwb.device.Device at 0x10977868304\nFields:\n description: 96 channel utah array\n manufacturer: BlackRock Microsystems\n\n location: Caudal, dorsal Pre-motor cortex, Left hemisphere\n",1,1.0,0.0
2,NaN,NaN,NaN,-1.0,"Pre-Motor Cortex, dorsal",1000Hz,"1 pynwb.ecephys.ElectrodeGroup at 0x11595178320\nFields:\n description: array corresponding to device implanted at PMd\n device: Utah Array(PMd) pynwb.device.Device at 0x10977868304\nFields:\n description: 96 channel utah array\n manufacturer: BlackRock Microsystems\n\n location: Caudal, dorsal Pre-motor cortex, Left hemisphere\n",1,1.0,0.0
3,NaN,NaN,NaN,-1.0,"Pre-Motor Cortex, dorsal",1000Hz,"1 pynwb.ecephys.ElectrodeGroup at 0x11595178320\nFields:\n description: array corresponding to device implanted at PMd\n device: Utah Array(PMd) pynwb.device.Device at 0x10977868304\nFields:\n description: 96 channel utah array\n manufacturer: BlackRock Microsystems\n\n location: Caudal, dorsal Pre-motor cortex, Left hemisphere\n",1,1.0,0.0


In [54]:
orig_trialtable = nwb.trials.to_dataframe()

In [57]:
orig_trialtable['target_size'].drop_duplicates()

id
0    13
Name: target_size, dtype: uint8

In [70]:
orig_trialtable['maze_num_barriers'].drop_duplicates()

id
0     2
2     9
6     7
8     0
12    6
16    8
Name: maze_num_barriers, dtype: uint8

In [59]:
orig_trialtable['barrier_info'].drop_duplicates()

id
0       [[-16.0, 96.0, 59.0, 22.0], [-62.0, -112.0, 43...
1       [[-49.0, -59.0, 22.0, 53.0], [-33.0, -100.0, 5...
2       [[-33.0, 47.0, 6.0, 37.0], [-77.0, 48.0, 11.0,...
3       [[-82.0, -43.0, 8.0, 118.0], [145.0, -105.0, 1...
4       [[-9.0, 52.0, 8.0, 43.0], [-50.0, 91.0, 64.0, ...
                              ...                        
2631    [[-64.0, -47.0, 113.0, 7.0], [-64.0, -39.0, 40...
2632    [[-64.0, -47.0, 113.0, 7.0], [-64.0, -39.0, 40...
2633    [[-9.0, 52.0, 8.0, 43.0], [-50.0, 91.0, 64.0, ...
2634    [[-69.0, -16.0, 69.0, 13.0], [-120.0, -62.0, 1...
2635    [[-33.0, 47.0, 6.0, 37.0], [-77.0, 48.0, 11.0,...
Name: barrier_info, Length: 2636, dtype: object

In [66]:
orig_trialtable.columns

Index(['start_time', 'stop_time', 'target_presentation_time', 'go_cue_time',
       'reaction_time', 'move_begins_time', 'move_ends_time', 'discard_trial',
       'task_success', 'trial_type', 'trial_version', 'proto_trial',
       'maze_condition', 'correct_reach', 'maze_num_targets',
       'maze_num_barriers', 'novel_maze', 'target_positions', 'frame_details',
       'hit_target_position', 'target_size', 'barrier_info', 'timeseries'],
      dtype='object')

In [68]:
orig_trialtable

,start_time,stop_time,target_presentation_time,go_cue_time,reaction_time,move_begins_time,move_ends_time,discard_trial,task_success,trial_type,...,correct_reach,maze_num_targets,maze_num_barriers,novel_maze,target_positions,frame_details,hit_target_position,target_size,barrier_info,timeseries
id,,,,,,,,,,,,,,,,,,,,,
0,4.707200,8.567133,5.570200,6.517200,0.335,6.819200,7.234200,0,1,0,...,0,1,2,1,"[[-130, -57]]","[[[-200]], [[200]], [[-155]], [[155]], [[20]]]","[-130, -57]",13,"[[-16.0, 96.0, 59.0, 22.0], [-62.0, -112.0, 43...","[(0, 3226, Eye pynwb.behavior.SpatialSeries at..."
1,9.407200,12.112167,10.172200,10.202200,0.305,10.479200,10.789200,0,1,0,...,0,1,2,1,"[[120, 70]]","[[[-200]], [[200]], [[-155]], [[155]], [[20]]]","[120, 70]",13,"[[-49.0, -59.0, 22.0, 53.0], [-33.0, -100.0, 5...","[(3226, 2081, Eye pynwb.behavior.SpatialSeries..."
2,18.437233,21.972267,19.209233,19.857233,0.300,20.124233,20.584233,0,1,17,...,1,1,9,0,"[[-77, 82]]","[[[-200]], [[200]], [[-155]], [[155]], [[20]]]","[-77, 82]",13,"[[-33.0, 47.0, 6.0, 37.0], [-77.0, 48.0, 11.0,...","[(5307, 2846, Eye pynwb.behavior.SpatialSeries..."
3,22.777233,26.007267,23.495233,23.959233,0.305,24.229233,24.644233,0,1,8,...,1,1,9,0,"[[132, 56]]","[[[-200]], [[200]], [[-155]], [[155]], [[20]]]","[132, 56]",13,"[[-82.0, -43.0, 8.0, 118.0], [145.0, -105.0, 1...","[(8153, 2566, Eye pynwb.behavior.SpatialSeries..."
4,26.937267,30.317267,27.630267,28.211267,0.300,28.479267,28.979267,0,1,29,...,1,1,9,0,"[[2, 82]]","[[[-200]], [[200]], [[-155]], [[155]], [[20]]]","[2, 82]",13,"[[-9.0, 52.0, 8.0, 43.0], [-50.0, 91.0, 64.0, ...","[(10719, 2741, Eye pynwb.behavior.SpatialSerie..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2631,15586.530559,15590.185559,15587.318559,15587.932559,0.315,15588.212559,15588.807559,0,1,13,...,1,1,8,0,"[[132, 56]]","[[[-200]], [[200]], [[-155]], [[155]], [[20]]]","[132, 56]",13,"[[-64.0, -47.0, 113.0, 7.0], [-64.0, -39.0, 40...","[(7990376, 2976, Eye pynwb.behavior.SpatialSer..."
2632,15591.780559,15594.615592,15592.688559,15593.202559,0.360,15593.532559,NaN,0,0,13,...,0,1,8,0,"[[132, 56]]","[[[-200]], [[200]], [[-155]], [[155]], [[20]]]","[132, 56]",13,"[[-64.0, -47.0, 113.0, 7.0], [-64.0, -39.0, 40...","[(7993352, 2831, Eye pynwb.behavior.SpatialSer..."
2633,15599.680592,15602.685592,15600.425592,15600.488592,0.365,15600.822592,15601.287592,0,1,29,...,1,1,9,0,"[[2, 82]]","[[[-200]], [[200]], [[-155]], [[155]], [[20]]]","[2, 82]",13,"[[-9.0, 52.0, 8.0, 43.0], [-50.0, 91.0, 64.0, ...","[(7996183, 2306, Eye pynwb.behavior.SpatialSer..."


In [17]:
### For now, let's do this kust for the erroe trials.

In [23]:
trialtable = orig_trialtable[orig_trialtable['task_success'] == 0]

In [25]:
trialtable

,start_time,stop_time,target_presentation_time,go_cue_time,reaction_time,move_begins_time,move_ends_time,discard_trial,task_success,trial_type,...,correct_reach,maze_num_targets,maze_num_barriers,novel_maze,target_positions,frame_details,hit_target_position,target_size,barrier_info,timeseries
id,,,,,,,,,,,,,,,,,,,,,
193,962.781200,966.041267,963.482200,964.413200,0.340,964.713200,965.193200,0,0,4,...,0,1,8,0,"[[132, 56]]","[[[-200]], [[200]], [[-155]], [[155]], [[20]]]","[132, 56]",13,"[[-64.0, -47.0, 113.0, 7.0], [-64.0, -39.0, 40...","[(569518, 3261, Eye pynwb.behavior.SpatialSeri..."
241,1193.081800,1196.051700,1193.843800,1194.541800,0.315,1194.823800,1195.123800,0,0,4,...,0,1,0,0,"[[132, 56]]","[[[-200]], [[200]], [[-155]], [[155]], [[20]]]","[132, 56]",13,[],"[(718796, 2970, Eye pynwb.behavior.SpatialSeri..."
450,2241.204467,2244.804500,2242.053467,2243.001467,0.340,2243.301467,NaN,0,0,4,...,0,1,8,0,"[[132, 56]]","[[[-200]], [[200]], [[-155]], [[155]], [[20]]]","[132, 56]",13,"[[-64.0, -47.0, 113.0, 7.0], [-64.0, -39.0, 40...","[(1352130, 3596, Eye pynwb.behavior.SpatialSer..."
694,3462.367800,3466.722467,3463.264800,3464.261800,0.325,3465.439800,3465.789800,0,0,1,...,0,1,8,0,"[[132, 56]]","[[[-200]], [[200]], [[-155]], [[155]], [[20]]]","[132, 56]",13,"[[-64.0, -47.0, 113.0, 7.0], [-64.0, -39.0, 40...","[(2069719, 4355, Eye pynwb.behavior.SpatialSer..."
824,4181.939500,4184.494533,4182.797500,4182.994500,0.460,4183.411500,NaN,0,0,11,...,0,3,8,0,"[[123, -81], [-130, -13], [123, 71]]","[[[-200]], [[200]], [[-155]], [[155]], [[20]]]","[123, 71]",13,"[[-65.0, -15.0, 51.0, 14.0], [-79.0, -55.0, 6....","[(2470824, 2551, Eye pynwb.behavior.SpatialSer..."
1012,5158.936833,5162.561733,5159.805833,5159.835833,0.325,5161.083833,5161.453833,0,0,1,...,0,1,8,0,"[[131, -55]]","[[[-200]], [[200]], [[-155]], [[155]], [[20]]]","[131, -55]",13,"[[-64.0, -47.0, 113.0, 7.0], [-64.0, -39.0, 40...","[(3037787, 3601, Eye pynwb.behavior.SpatialSer..."
1072,5451.552767,5455.473200,5452.231767,5452.928767,0.875,5453.764767,NaN,0,0,1,...,0,1,8,0,"[[131, -55]]","[[[-200]], [[200]], [[-155]], [[155]], [[20]]]","[131, -55]",13,"[[-64.0, -47.0, 113.0, 7.0], [-64.0, -39.0, 40...","[(3220137, 3916, Eye pynwb.behavior.SpatialSer..."
1154,5921.413833,5924.343867,5922.092833,5922.790833,0.410,5923.155833,NaN,0,0,7,...,0,3,9,0,"[[133, -81], [-118, -83], [-77, 82]]","[[[-200]], [[200]], [[-155]], [[155]], [[20]]]","[-77, 82]",13,"[[-33.0, 47.0, 6.0, 37.0], [-77.0, 48.0, 11.0,...","[(3469594, 2926, Eye pynwb.behavior.SpatialSer..."
1475,7561.802431,7565.272331,7562.463431,7562.543431,0.380,7563.774431,7564.094431,0,0,1,...,0,1,8,0,"[[131, -55]]","[[[-200]], [[200]], [[-155]], [[155]], [[20]]]","[131, -55]",13,"[[-64.0, -47.0, 113.0, 7.0], [-64.0, -39.0, 40...","[(4430270, 3376, Eye pynwb.behavior.SpatialSer..."


In [162]:
trialtable = nwb.trials.to_dataframe()
neurons = nwb.units.to_dataframe()['spike_times'] # Neuron id x Spike Time
time = np.round(nwb.processing['behavior'].data_interfaces['Position'].spatial_series['Hand'].timestamps[:]+ 0.0001,decimals=3)
hand = nwb.processing['behavior'].data_interfaces['Position'].spatial_series['Hand'].data[:] # x, y 
cursor = nwb.processing['behavior'].data_interfaces['Position'].spatial_series['Cursor'].data[:] # x, y
gocue = np.round(trialtable['go_cue_time'],decimals=3)
startmov = np.round(trialtable['move_begins_time'], decimals=3)
endmov = np.round(trialtable['move_ends_time'], decimals=3)
reactiontime = np.round(trialtable['reaction_time'], decimals=3)
target_pos = trialtable['target_positions']
num_neurons = neurons.shape[0]
num_trials = len(trialtable)

trial_type = trialtable['trial_type']
trial_version = trialtable['trial_version']
is_cor_reach = trialtable['correct_reach']
task_success = trialtable['task_success']
maze_conds = trialtable['maze_condition']
start_time = np.round(trialtable['start_time'], decimals=3)
stop_time = np.round(trialtable['stop_time'], decimals=3)
targeton = np.round(trialtable['target_presentation_time'], decimals=3)
hit_target_pos = trialtable['hit_target_position']

In [72]:
data_list = {'firing_rates': [],
             'events': [],
             'hand': [],
             'cursor': [],
             'result': [],
             'maze_num_targets': [],
             'maze_num_barriers': [],
             'target_positions':[],
             
             'trial_type': [],
             'trial_version': [],
             'correct_reach': [],
             'task_success': [],
             'maze_condition': [],
             'start_time': [],
             'stop_time': [],
             'target_on_time': [],
             'hit_target_position': []}


for trial_id in tqdm(range(0, num_trials)):

            min_T = np.round(trialtable['start_time'][trial_id],decimals=3) # Trial start time
            max_T = np.round(trialtable['stop_time'][trial_id],decimals=3) # Trial end time

            mi = np.nonzero(time>min_T)[0][0]
            ma = np.nonzero(time<max_T)[0][-1]

            grid = np.arange(min_T, max_T + 0.001, 0.001) #seconds
            grid = np.round(grid, decimals=3)
            grids = grid[:-1]
            gride = grid[1:]
            num_bins = len(grid)

            neurons_binned = np.zeros((num_bins, num_neurons))
            hand_binned = np.zeros((num_bins, 2))
            cursor_binned = np.zeros((num_bins, 2))
            targets_binned = np.zeros((num_bins,))
            events_binned = np.zeros((num_bins,))
            result= np.zeros((num_bins,))
            maze_barriers = np.zeros((num_bins,))
            maze_targets = np.zeros((num_bins,))
            reaction_t = np.zeros((num_bins,))
            targets_pos = np.zeros((num_bins,2))
            
            target_size = np.zeros((num_bins,))
            trial_type = np.zeros((num_bins,))
            trial_version = np.zeros((num_bins,))
            correct_reach = np.zeros((num_bins,))
            task_success = np.zeros((num_bins,))
            #maze_condition = np.zeros((num_bins,))
            #start_time = np.zeros((num_bins,))
            #stop_time = np.zeros((num_bins,))
            #target_on_time = np.zeros((num_bins,))
            #hit_target_pos = np.zeros((num_bins, 2))
            
            #targets_dir += target_dir_degree #target dir

            _, hand_binned[:,0] = np.histogram(hand[mi:ma,0], bins=num_bins-1)
            _, hand_binned[:,1] = np.histogram(hand[mi:ma,1], bins=num_bins-1)

            _, cursor_binned[:,1] = np.histogram(cursor[mi:ma,1], bins=num_bins-1)
            _, cursor_binned[:,1] = np.histogram(cursor[mi:ma,1], bins=num_bins-1)

            #hand_binned = hand[mi:ma]
            #cursor_binned = cursor[mi:ma]
            
            maze_barriers += trialtable['maze_num_barriers'][trial_id]
            targets_pos += target_pos[trial_id][0]
            reaction_t += reactiontime[trial_id]

            maze_targets += trialtable['maze_num_targets'][trial_id]

            result += trialtable['task_success'][trial_id]
            
            hit_target_pos += trialtable['hit_target_position'][trial_id]

            for n in range(num_neurons):
                # Rounding the spiketimes to two decimal places
                rounded_spike_times = np.round(neurons[n], decimals=3)

                neurons_binned[:, n] = np.isin(grid, rounded_spike_times).astype(int)
                
            if gocue[trial_id] > 0 and gocue[trial_id] <= grid[-1]:

                idx_go = np.where(grid == gocue[trial_id])[0][0]
                events_binned[idx_go] = 2 #go cue

            if startmov[trial_id] > 0:

                idx_startmov = np.where(grid == startmov[trial_id])[0][0]
                events_binned[idx_startmov] = 3 #move start

            if endmov[trial_id] > 0:
                idx_endmov = np.where(grid == endmov[trial_id])[0][0]
                events_binned[idx_endmov] = 4 #move end
                
            if targeton[trial_id] > 0 and targeton[trial_id] <= grid[-1]:

                idx_targeton = np.where(grid == targeton[trial_id])[0][0]
                events_binned[idx_targeton] = 5 #target on
                
            if start_time[trial_id] > 0 and start_time[trial_id] <= grid[-1]:

                idx_start = np.where(grid == start_time[trial_id])[0][0]
                events_binned[idx_start] = 6 #start
                
            if stop_time[trial_id] > 0 and stop_time[trial_id] <= grid[-1]:

                idx_stop = np.where(grid == stop_time[trial_id])[0][0]
                events_binned[idx_stop] = 7 #end

            data_list['firing_rates'].append(neurons_binned)
            data_list['cursor'].append(cursor_binned)
            data_list['hand'].append(hand_binned)
            data_list['events'].append(events_binned)
            data_list['result'].append(result)
            data_list['maze_num_barriers'].append(maze_barriers)
            data_list['maze_num_targets'].append(maze_targets)
            data_list['target_positions'].append(targets_pos)
            
            data_list['trial_type'].append()
            data_list['trial_version'].append()
            data_list['correct_reach'].append()
            data_list['task_success'].append()
            data_list['maze_condition'].append()
            data_list['start_time'].append()
            data_list['stop_time'].append()
            data_list['target_on_time'].append()
            data_list['hit_target_position'].append()

  0%|          | 5/2636 [00:15<2:18:22,  3.16s/it]


KeyboardInterrupt: 

In [169]:
events_binned.shape

(4177,)

In [ ]:
np.where(grid == gocue[trial_id])

In [41]:
grid

array([4.707, 4.708, 4.709, ..., 8.565, 8.566, 8.567])

In [49]:
gocue[trial_id][0]

IndexError: invalid index to scalar variable.

In [ ]:
data_list.keys()

In [ ]:
fr = pd.DataFrame(np.vstack(data_list["firing_rates"]), columns=["Neuron"+str(i) for i in range(1, num_neurons+1)])
fr['trial_id'] = np.repeat(range(0, num_trials), [arr.shape[0] for arr in data_list['firing_rates']])

In [ ]:
fr

In [ ]:
h = pd.DataFrame(np.vstack(data_list["hand"]), columns=["hand_pos_x", "hand_pos_y"])
c = pd.DataFrame(np.vstack(data_list["cursor"]), columns=["cursor_pos_x", "cursor_pos_y"])
tpos = pd.DataFrame(np.vstack(data_list["target_positions"]), columns=["target_pos_x", "target_pos_y"])

In [ ]:
tpos

In [ ]:
c

In [ ]:
h

In [ ]:
numtargets = pd.DataFrame(np.hstack(data_list["maze_num_targets"]), columns=['maze_num_target'])
numb = pd.DataFrame(np.hstack(data_list["maze_num_barriers"]), columns=['maze_num_barriers'])

In [ ]:
numtargets

In [ ]:
numb

In [ ]:
event = pd.DataFrame(np.hstack(data_list["events"]), columns=['events'])

In [ ]:
event

In [ ]:
r = pd.DataFrame(np.hstack(data_list["result"]), columns=['result'])

In [ ]:
r

In [ ]:
ev = pd.concat([h,c,tpos,numtargets,numb,event,r], axis=1)

In [ ]:
result = pd.concat([fr,ev], axis=1)

In [ ]:
result

In [ ]:
fr['Neuron1'].describe()

In [ ]:
event['events'].value_counts()

In [ ]:
result

In [ ]:
# Save the DataFrame to a Parquet file
parquet_file_path = 'processed_data/Churchland/sub-Nitschke_ses-20090920_behavior+ecephys.parquet' 
table = pa.Table.from_pandas(result)
pq.write_table(table, parquet_file_path)